In [1]:
!pip install --upgrade keras
!pip install opencv-python

    100% |████████████████████████████████| 307kB 5.0MB/s 
    100% |████████████████████████████████| 51kB 9.6MB/s 
  Found existing installation: Keras 2.1.6
    Uninstalling Keras-2.1.6:
      Successfully uninstalled Keras-2.1.6


In [0]:
import keras
from keras.models import Model
from keras.layers import Input, UpSampling2D, MaxPool2D, Activation
from keras.layers.convolutional import Conv2D
from keras.layers.merge import Add
from keras.optimizers import Adam
from keras.datasets.cifar10 import load_data
import cv2
import numpy as np

In [0]:
class DDSRCNN(object):
  def __init__(self, rows=32, cols=32, channels=3):
    self.rows = rows
    self.cols = cols
    self.channels = channels
    self.model = self._make_model()
    optimizer = Adam(lr=0.0001, beta_1=0.1)
    self.model.compile(loss="mse", optimizer=optimizer)
    
  def _make_model(self):
    image_shape  = (self.rows, self.cols, self.channels)
    inputs = Input(shape=image_shape)
    conv1 = Conv2D(filters=64, kernel_size=(3,3), padding="same")(inputs)
    conv1 = Conv2D(filters=64, kernel_size=(3,3), padding="same")(conv1)
    
    # conv2
    conv2 = MaxPool2D(pool_size=(2,2))(conv1)
    conv2 = Conv2D(filters=128, kernel_size=(3,3), padding="same")(conv2)
    conv2 = Conv2D(filters=128, kernel_size=(3,3), padding="same")(conv2)
    
    # conv3
    conv3 = MaxPool2D(pool_size=(2,2))(conv2)
    conv3 = Conv2D(filters=256, kernel_size=(3,3), padding="same")(conv3)
    
    out = UpSampling2D(size=(2,2))(conv3)
    out = Conv2D(filters=128, kernel_size=(3,3), padding="same")(out)
    out = Conv2D(filters=128, kernel_size=(3,3), padding="same")(out)
    out = Add()([out, conv2])
    out = Activation("relu")(out)
    out = UpSampling2D(size=(2,2))(out)
    out = Conv2D(filters=64, kernel_size=(3,3), padding="same")(out)
    out = Conv2D(filters=64, kernel_size=(3,3), padding="same")(out)
    out = Add()([out, conv1])
    out = Activation("relu")(out)
    out = Conv2D(filters=self.channels, kernel_size=(3,3), padding="same")(out)
    
    model = Model(inputs=inputs, outputs=out)
    model.summary()
    return model
    
  def train(self, epochs=50):
    (X_train, _), (X_test, _) = load_data()    
    X_train_noise = [cv2.GaussianBlur(img, ksize=(3,3), sigmaX=1.3) for img in X_train]
    X_train_noise = np.array(X_train_noise)
    X_train_noise = (X_train_noise.astype(np.float32) - 127.5) / 127.5
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    self.model.fit(X_train_noise, X_train, epochs=epochs, validation_split=0.1, batch_size=128)
    self.model.save("DDSRCNN.h5")
    from google.colab import files
    files.download("DDSRCNN.h5") 

In [45]:
model = DDSRCNN()
model.train(epochs=14)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_121 (Conv2D)             (None, 32, 32, 64)   1792        input_14[0][0]                   
__________________________________________________________________________________________________
conv2d_122 (Conv2D)             (None, 32, 32, 64)   36928       conv2d_121[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_26 (MaxPooling2D) (None, 16, 16, 64)   0           conv2d_122[0][0]                 
__________________________________________________________________________________________________
conv2d_123

45000/45000 [==============================] - 48s 1ms/step - loss: 0.0206 - val_loss: 0.0095
Epoch 2/14
45000/45000 [==============================] - 47s 1ms/step - loss: 0.0080 - val_loss: 0.0069
Epoch 3/14
 6784/45000 [===>..........................] - ETA: 38s - loss: 0.0068

45000/45000 [==============================] - 47s 1ms/step - loss: 0.0063 - val_loss: 0.0059
Epoch 4/14
45000/45000 [==============================] - 47s 1ms/step - loss: 0.0073 - val_loss: 0.0053
Epoch 5/14
 7936/45000 [====>.........................] - ETA: 37s - loss: 0.0054

45000/45000 [==============================] - 47s 1ms/step - loss: 0.0052 - val_loss: 0.0049
Epoch 6/14
45000/45000 [==============================] - 47s 1ms/step - loss: 0.0048 - val_loss: 0.0046
Epoch 7/14
 8192/45000 [====>.........................] - ETA: 37s - loss: 0.0046

45000/45000 [==============================] - 47s 1ms/step - loss: 0.0045 - val_loss: 0.0043
Epoch 8/14
45000/45000 [==============================] - 47s 1ms/step - loss: 0.0042 - val_loss: 0.0040
Epoch 9/14
 8320/45000 [====>.........................] - ETA: 37s - loss: 0.0043

45000/45000 [==============================] - 47s 1ms/step - loss: 0.0039 - val_loss: 0.0037
Epoch 10/14
45000/45000 [==============================] - 47s 1ms/step - loss: 0.0037 - val_loss: 0.0034
Epoch 11/14
 8064/45000 [====>.........................] - ETA: 37s - loss: 0.0035

45000/45000 [==============================] - 47s 1ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 12/14
45000/45000 [==============================] - 47s 1ms/step - loss: 0.0031 - val_loss: 0.0031
Epoch 13/14
 8064/45000 [====>.........................] - ETA: 37s - loss: 0.0031

45000/45000 [==============================] - 47s 1ms/step - loss: 0.3267 - val_loss: 0.0420
Epoch 14/14
45000/45000 [==============================] - 47s 1ms/step - loss: 0.0169 - val_loss: 0.0108


MessageError: ignored

In [0]:
from google.colab import files

In [0]:
files.download("./DDSRCNN.h5
               ")